In [ ]:
import tensorflow as tf
import keras 
print(tf.__version__)
print(keras.__version__)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator ,load_img

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import random
import pathlib

In [ ]:
#Importing Data from CSV file
fer2013=pd.read_csv("../input/fer2013/fer2013.csv")

In [ ]:
labels=fer2013.iloc[:,[0]].values

In [ ]:
pixels=fer2013['pixels']

In [ ]:
#Facial Expressions
Ekspresi={0:"Marah",1:"Jijik",2:"Takut",3:"Senang",4:"Sedih",5:"Terkejut",6:"Netral"}
from keras.utils import to_categorical 
labels = to_categorical(labels,len(Ekspresi))

In [ ]:
#converting pixels to Gray Scale images of 48X48 
image = np.array([np.fromstring(pixel, dtype=int, sep=" ")for pixel in pixels])
image = image/255.0
image = image.reshape(image.shape[0],48,48,1).astype('float32')

In [ ]:
plt.imshow(image[0][:,:,0])
Ekspresi[labels[0][0]]

In [ ]:
#splitting data into training and test data
train_features,test_features,train_labels,test_labels = train_test_split(image,labels,test_size=0.2,random_state=0)

In [ ]:
train_labels

In [ ]:
def cnn_model(classes):
    model = Sequential()
    model.add(Conv2D(32,kernel_size=(3,3),strides=(1,1),activation='relu',input_shape=(48,48,1)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),strides=(1,1)))
    model.add(Dropout(0.25))#to prevent neural network from overfitting
    
    model.add(Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),strides=(1,1)))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2,2),strides=(1,1)))
    model.add(Dropout(0.25))
    
    model.add(Flatten())
    
    model.add(Dense(256,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    
    model.add(Dense(512,activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.25))
    
    model.add(Dense(classes,activation='softmax'))
    
    
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model


In [ ]:
classes=7
model = cnn_model(classes)
model.summary()

In [ ]:
#train the CNN 
model.fit(train_features,train_labels,batch_size=105,epochs=40,verbose=2)

In [ ]:
# save model untuk digunakan menguji model
model.save("model.h5")
print("Saved model to disk")

In [ ]:
# Membuat model tflite
export_dir = 'saved_model/1'

tf.saved_model.save(model, export_dir)

In [ ]:
# Select mode of optimization
mode = "Speed" 

if mode == 'Storage':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_SIZE
elif mode == 'Speed':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
else:
    optimization = tf.lite.Optimize.DEFAULT

In [ ]:

# Use the TFLiteConverter SavedModel API to initialize the converter

converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)

# Set the optimzations
converter.optimizations = [optimization]

# Invoke the converter to finally generate the TFLite model
tflite_model = converter.convert()

In [ ]:
# Save model tflitr
tflite_model_file = pathlib.Path('./model.tflite')
tflite_model_file.write_bytes(tflite_model)